# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. 

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Предобработка данных не понадобится — мы её уже сделали **в рамках проекта по определению ВЫГОДНОГО ТАРИФА ДЛЯ ТЕЛЕКОМ КОМПАНИИ**.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

## Загрузка и первичный осмотр данных

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных**

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Можно сделать вывод о том, что нам предоставлены данные без пропущенных значений и сильных выбросов, с корректными наименованиями столбцов, по которым к ним легко будет обращаться. Нет слишком больших, а также отрицательных значений, которые не вписались бы в логику по смыслу информации, хранящейся в столбцах. В столбце нет is_ultra (тариф) нет лишних тарифов, кроме имеющихся в описании: только «Ультра» — 1, «Смарт» — 0.

## Разбиение данных на выборки

Перед нами стоит задача построить модель для задачи классификации, которая выберет подходящий тариф.
Целевой признак в задаче классификации является категориальным. У нас целевой признак - это тариф (Ультра или Смарт) и он категориальный.

In [6]:
df.shape

(3214, 5)

Разобьем весь набор данных на 2 выборки: обучающую и другую (выделим на вторую выборку 40%, а параметр random_state приравняем к любому значению (кроме None) - чтобы зафиксировать псевдослучайность):

In [7]:
df_train, df_test_valid = train_test_split(df, test_size=0.4, random_state=12345, stratify=df.is_ultra) 

print(df_train.shape)
print(df_test_valid.shape)

(1928, 5)
(1286, 5)


Разобьем вторую получившуюся выборку пополам еще на 2 выборки: тестовую и валидационную (укажем параметр test_size=0.5, чтобы определить размер валидационной выборки как 50%)

In [8]:
df_test, df_valid = train_test_split(df_test_valid, test_size=0.5, random_state=12345, stratify=df_test_valid.is_ultra)

print(df_test.shape)
print(df_valid.shape)

(643, 5)
(643, 5)


В итоге, мы получили 3 выборки:
* тренировочную df_train (60%)
* валидационную df_valid (20%)
* тестовую df_test (20%)

Объявим переменные с признаками для обучения:  
target_train (цель для обучения) и  features_train (признаки для обучения):

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


Создадим аналогичные переменные для проверки:  
с целевым признаком — target_test и target_valid  
с остальными признаками — features_test и features_valid

In [10]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


In [11]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


По выводам размеров таблиц видим, что все поделено корректно.

## Исследование моделей

Для решения задачи классификации, мы изучили 3 модели: дерево решений, случайный лес и логистическая регрессия.  
Исследуем перечисленные модели на наших данных.


**Дерево решений**

Воспользуемся DecisionTreeClassifier из библиотеки sklearn.tree.  

* Чтобы найти лучшее значение гиперпараметра maх_depth, переберем в цикле разные его значения и сравним качество моделей
* Запишем модель в переменной, указав гиперпараметры. Зафиксируем псевдослучайность параметром random_state
* Чтобы управлять глубиной вложенности дерева регений, пропишем гиперпараметр max_depth
* Обучим модель методом fit() на тренировочной выборке, в результате модель будет обучена и готова предсказывать
* Вызовем метод predict() для предсказания результатов


In [12]:
best_model_tree = None
best_result = 0

for depth in range(1, 5):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    
    if result > best_result:
        best_model_tree = model_tree
        best_result = result
        best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Оптимальная глубина вложенности дерева решений:", best_depth)    

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Оптимальная глубина вложенности дерева решений: 3


При использовании модели дерево решений, оптимальной глубиной дерева оказалась max_depth = 3, т.к. у нее самое высокое значение accuracy =  0.799, в сравнении с другими найденными в цикле.

**Случайный лес**

Используем алгоритм случайного леса RandomForestClassifier из библиотеки sklearn.ensemble.

* Запишем модель в переменной, указав гиперпараметры. Зафиксируем псевдослучайность параметром random_state
* Чтобы управлять количеством деревьев в лесу, пропишем гиперпараметр n_estimator
* Обучим модель на тренировочной выборке методом .fit()
* Проверим качество модели методом score(), который производит подсчет accuracy для всех алгоритмов классификации.
* В best_model сохраним наилучшую модель
* В best_result сохраним наилучшее значение метрики accuracy на валидационных данных и распечатаем его

In [13]:
best_model_forest = None
best_result = 0

for est in range(1, 11):
    
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    result = model_forest.score(features_valid, target_valid)
    
    if result > best_result:
        best_model_forest = model_forest
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Оптимальное количество деревьев в лесу:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Оптимальное количество деревьев в лесу: 9


Получившееся значение accuracy чуть меньше чем у дерева решений, значит модель "случайный лес" поможет нам получить менее точные предсказания на валидационной выборке, чем модель "дерево решений", однако значения почти равны (отличие на 5 тысячных).

**Логистическая регрессия**

Воспользуемся моделью LogisticRegression из модуля sklearn.linear_model

* Запишем модель в переменной model, указав гиперпараметры. Для постоянства результата зададим random_state = 12345
* Обучим модель на тренировочной выборке вызовом метода fit() с параметрами features_train, target_train
* Вызовем функцию .score(), чтобы получить метрику качества предсказаний модели (accuracy) на валидационной выборке 

In [14]:
model_regr = LogisticRegression(random_state=12345)
model_regr.fit(features_train, target_train)
result = model_regr.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


В результате применения логистической регрессии мы получили самое низкое значение метрики качества предсказаний на валидационной выборке, а значит данная модель предскажет хуже чем "Случайный лес" и "Дерево решений".

## Тестирование моделей

**Дерево решений**  
Произведем предсказания на тестовой выборке и посмотрим на количество ошибок:

In [15]:
predictions = list(best_model_tree.predict(features_test))
answers = list(target_test)

def error_count(answers, predictions):
    n = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            n += 1
    return n

print("Ошибок:", error_count(answers, predictions))

Ошибок: 143


In [16]:
len(answers)

643

Посмотрим на accuracy на тестовой выборке:

In [17]:
test_predictions = best_model_tree.predict(features_test)
accuracy_test = accuracy_score(target_test, test_predictions) 

print("Accuracy на тестовой выборке (Дерево решений):", accuracy_test)

Accuracy на тестовой выборке (Дерево решений): 0.7776049766718507


На валидационной выборке модель предсказывает немного лучше, но, можно сказать, что почти одинаково.


**Случайный лес**  
Проверим то же самое на сохраненной модели случайного леса:

In [18]:
test_predictions = best_model_forest.predict(features_test)
accuracy_test = accuracy_score(target_test, test_predictions) 

print("Accuracy на тестовой выборке (Случайный лес)", accuracy_test)

Accuracy на тестовой выборке (Случайный лес) 0.7698289269051322


Accuracy на тестовой выборке ниже, чем на валидационной, и почти равно предсказанию по модели Дерева решений.

**Логистическая регрессия**

In [19]:
test_predictions = model_regr.predict(features_test)
accuracy_test = accuracy_score(target_test, test_predictions) 

print("Accuracy на тестовой выборке (Логистическая регрессия)", accuracy_test)

Accuracy на тестовой выборке (Логистическая регрессия) 0.71850699844479


Предсказания на валидационной и тестовой выборках сильно похожи, но являются самыми отдаленными от реальности среди всех остальных способов: accuracy не достиг даже 0,75

Можно сделать вывод о том, что алгоритмы случайного леса и дерева решений помогли сделать нам самые близкие к реальным показателям предсказания, но если учесть что скорость выполнения у алгоритмов разная, то решающее дерево выигрывает, т.к. оно выполняется быстрее.

## Проверка модели на адекватность

Результаты предыдущего пункта свидетельствуют о том, что мы можем применять модели машинного обучения такие как "Дерево решений" и "Случайный лес" для прогнозирования тарифа на основе доступных данных.  
С помощью "Дерева решений" удалось на тестовых данных добиться погрешности в пределах 0.045. С учетом высокой скорости выполнения и относительно невысокой погрешности, данная модель может считаться самой адекватной.

Исследуем модели на адекватность при помощи DummyClassifier из библиотеки sklearn.dummy:

In [26]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy="most_frequent").fit(features_train, target_train)
dummy_pred = dummy.predict(target_test)
dummy_score = accuracy_score(target_test, dummy_pred)

print("accuracy на тестовом наборе:", dummy_score)

accuracy на тестовом наборе: 0.6936236391912908


Accuracy всех рассматриваемых нами выше моделей оказался больше чем dummy_score = 0,69.  
Таким образом, делаем вывод о том, что все вышеуказанные модели являются адекватными.